In [ ]:
from selenium import webdriver
from webdriver_manager import driver
from webdriver_manager.chrome import ChromeDriver, ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from time import sleep
from bs4 import BeautifulSoup

driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get("https://roll20.net/compendium/dnd5e/Monsters%20List#content")
page = driver.page_source

# Wait until the page is loaded
sleep(13)

soup = BeautifulSoup(page, "html.parser")

# Get all card urls
urls = soup.find_all("div", {"class":"header inapp-hide single-hide"})

cards_list = []
for url in urls:
    card = {}
    card_url = url.a["href"]
    card_link = f'https://roll20.net{card_url}#content'

    # Open chrom in headless mode
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)
    driver.get(card_link )
    page = driver.page_source
    soup = BeautifulSoup(page, "html.parser")
    card["name"] = soup.select_one(".name").text.split("\n")[1]
    card["subtitle"] = soup.select_one(".subtitle").text
    # Do the same to get all card detealis 

    cards_list.append(card)
    driver.quit()

In [2]:
import selenium
selenium.__path__

['/Users/rentsher/opt/anaconda3/lib/python3.9/site-packages/selenium']

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
import csv
import time
from bs4 import BeautifulSoup

try:
    # Set up Chrome WebDriver with webdriver-manager for Selenium 4
    chrome_service = ChromeService(ChromeDriverManager().install())

    # Initialize the WebDriver with the Chrome service
    driver = webdriver.Chrome(service=chrome_service)

    # Navigate to the specific URL
    url = "https://play.google.com/work/search?q=ott"
    driver.get(url)

    # Scroll to load more apps until we have collected as many URLs as possible in 1 minute
    start_time = time.time()  # Record the start time
    app_urls = set()

    while time.time() - start_time < 60:  # Run for 1 minute
        # Scroll down to load more apps
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait for a shorter time (adjust as needed) to load more content
        time.sleep(1)  # Wait for 1 second

    # Get the page source
    page_source = driver.page_source

    # Close the WebDriver
    driver.quit()

    # Parse the page source using BeautifulSoup
    soup = BeautifulSoup(page_source, 'html.parser')

    # Find all app elements with the specified class
    app_elements = soup.find_all('a', class_='b8cIId')

    # Extract app URLs and add them to the set
    for app_element in app_elements:
        app_url = app_element.get('href')
        if app_url and app_url.startswith("/work/apps/details?id="):
            app_url = "https://play.google.com" + app_url
            app_urls.add(app_url)

    # Save the app URLs to a CSV file locally
    csv_file_path = "/Users/rentsher/Desktop/playstore.csv"  # Update with your local file path

    with open(csv_file_path, "w", newline="") as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(["App URLs"])
        for app_url in app_urls:
            csv_writer.writerow([app_url])

    print(f"Saved {len(app_urls)} app URLs to {csv_file_path}")

except Exception as e:
    print(f"An error occurred: {str(e)}")


An error occurred: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=119.0.6045.159)
Stacktrace:
0   chromedriver                        0x0000000100edcd28 chromedriver + 4795688
1   chromedriver                        0x0000000100ed42b3 chromedriver + 4760243
2   chromedriver                        0x0000000100aad88d chromedriver + 407693
3   chromedriver                        0x0000000100a81758 chromedriver + 227160
4   chromedriver                        0x0000000100b2950f chromedriver + 914703
5   chromedriver                        0x0000000100b3f0e6 chromedriver + 1003750
6   chromedriver                        0x0000000100b23a73 chromedriver + 891507
7   chromedriver                        0x0000000100aee143 chromedriver + 672067
8   chromedriver                        0x0000000100aef31e chromedriver + 676638
9   chromedriver                        0x0000000100e9d795 chromedriver + 4536213
10  chromedriver      

In [30]:
import requests
import csv

# Define the API endpoint URL
api_url = "https://play.google.com/work/search/apps?hl=en&query=ott&num=50"  # Modify the query and num as needed
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
}

response = requests.get(api_url, headers=headers)

try:
    # Send a GET request to the API endpoint
    response = requests.get(api_url)

    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()

        # Extract app URLs from the API response
        app_urls = []
        for item in data.get('apps', []):
            app_id = item.get('package_name')
            if app_id:
                app_url = f"https://play.google.com/work/apps/details?id={app_id}"
                app_urls.append(app_url)

        # Save the app URLs to a CSV file locally
        csv_file_path = "/Users/rentsher/Desktop/playstore.csv"  # Update with your local file path

        with open(csv_file_path, "w", newline="") as csvfile:
            csv_writer = csv.writer(csvfile)
            csv_writer.writerow(["App URLs"])
            csv_writer.writerows([[app_url] for app_url in app_urls])

        print(f"Saved {len(app_urls)} app URLs to {csv_file_path}")

    else:
        print(f"Failed to retrieve data. Status code: {response.status_code}")

except Exception as e:
    print(f"An error occurred: {str(e)}")


Failed to retrieve data. Status code: 404


In [9]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import csv
from concurrent.futures import ThreadPoolExecutor
from urllib.parse import urljoin

# Function to fetch data from a given URL
def fetch_data(url):
    driver.get(url)
    
    # Wait for the page to load (you might need to adjust the wait time)
    driver.implicitly_wait(10)
    
    # Extract data from the specified div and anchor elements
    div_elements = driver.find_elements(By.CLASS_NAME, 'b8cIId')
    anchor_elements = driver.find_elements(By.CLASS_NAME, 'JC71ub')
    
    data_list = []
    
    # Iterate over the elements and extract data
    for div, anchor in zip(div_elements, anchor_elements):
        data = {
            'div_data': div.text,
            'anchor_href': urljoin(url, anchor.get_attribute('href'))
        }
        data_list.append(data)

    return data_list

# Set the URL and the number of threads
base_url = "https://play.google.com/work/search?q=TV%20Today%20Network%20Limited&c=apps"
num_threads = 5  # You can adjust this based on your requirements

# Initialize the webdriver
driver = webdriver.Chrome()

# Create a CSV file to store the data
csv_file = open('/Users/rentsher/Desktop/output_data.csv', 'w', newline='', encoding='utf-8')
fieldnames = ['div_data', 'anchor_href']
csv_writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

# Write the header to the CSV file
csv_writer.writeheader()

# Generate a list of URLs to scrape (you might need to adapt this based on the website structure)
url_list = [f"{base_url}/page/{i}" for i in range(1, 11)]  # Example: scraping pages 1 to 10

# Function to fetch data and write to CSV
def fetch_and_write_data(url):
    data = fetch_data(url)
    
    # Write the data to the CSV file
    csv_writer.writerows(data)

# Use ThreadPoolExecutor for concurrent execution
with ThreadPoolExecutor(max_workers=num_threads) as executor:
    executor.map(fetch_and_write_data, url_list)

# Close the CSV file and the webdriver
csv_file.close()
driver.quit()


Unable to obtain driver using Selenium Manager: Selenium Manager failed for: /Users/rentsher/opt/anaconda3/lib/python3.9/site-packages/selenium/webdriver/common/macos/selenium-manager --browser chrome --output json.
The chromedriver version cannot be discovered


SeleniumManagerException: Message: Selenium Manager failed for: /Users/rentsher/opt/anaconda3/lib/python3.9/site-packages/selenium/webdriver/common/macos/selenium-manager --browser chrome --output json.
The chromedriver version cannot be discovered
